In [1]:
#|default_exp img_sen2

# Sentinel 2

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee
import geemap
from datetime import datetime

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const
from gee_projects import mask_s2 as mask_s2

sentinel-2 class

In [16]:
#|export

class access_s2_image():
  
  def __init__(self,
               path_to_geometry : str , # path to geometry file, can be .shp or .gpkg . example : r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg"
               crs : str , #coordinate reference system to work with
               start_date : str , # Start date for searching images. , 
               end_date : str , # End date for searching images.
               user_date : str , # The user-provided date of interest; the script accesses the image with the closest available date to the provided date
               ):
    
    self.crs = crs
    self.fc = utils.gdf_to_featureCollection(
      utils.read_gdf(geometry_path=path_to_geometry,crs=self.crs))
    self.start_date=start_date
    self.end_date=end_date
    self.user_date = user_date
    
    
    self.fc = utils.gdf_to_featureCollection(utils.read_gdf
                                        (geometry_path=path_to_geometry,crs=self.crs))
    
    #pre process s2
    s2_sr_cld_col_eval = mask_s2.get_s2_sr_cld_col(self.fc, self.start_date, self.end_date)

    self.s2_sr = (s2_sr_cld_col_eval.map(mask_s2.add_cld_shdw_mask)
             .map(mask_s2.apply_cld_shdw_mask))

    #get list of available dates of the image collection

    self.available_dates = utils.list_dates(self.s2_sr)
    self.closest_date = utils.find_closest_date(date_list = self.available_dates,
                                                user_date=self.user_date)
    
    #filter image collection to get the image from the closest date :
    print(f'Requested date: {self.user_date}\n' 
          f'The nearest date available: {self.closest_date}')
    self.img = self.s2_sr.filterDate(self.closest_date, self.closest_date).mosaic().clip(self.fc)


In [17]:
extractor = access_s2_image(path_to_geometry=r"D:\git\gee_maps\nbs\geometry\test_bbox.gpkg", 
                            crs = "4326",
                            start_date = '2023-05-06',
                            end_date = '2023-10-08',
                            user_date = '2023-10-06')

extractor.img

Requested date: 2023-10-06
The nearest date available: 2023-10-07


In [ ]:
find_closest_date(date_list = test,user_date='2023-10-06')

In [ ]:
test=['2023-05-10', '2023-05-10', '2023-05-15', '2023-05-15', '2023-05-20', '2023-05-20', '2023-05-25', '2023-05-25', '2023-05-30', '2023-05-30', '2023-06-04', '2023-06-04', '2023-06-14', '2023-06-14', '2023-06-19', '2023-06-19', '2023-06-24', '2023-06-24', '2023-06-29', '2023-06-29', '2023-07-04', '2023-07-04', '2023-07-09', '2023-07-09', '2023-07-14', '2023-07-14', '2023-07-19', '2023-07-19', '2023-07-24', '2023-07-24', '2023-07-29', '2023-07-29', '2023-08-03', '2023-08-03', '2023-08-08', '2023-08-08', '2023-08-13', '2023-08-13', '2023-08-18', '2023-08-18', '2023-08-23', '2023-08-23', '2023-08-28', '2023-08-28', '2023-09-02', '2023-09-02', '2023-09-07', '2023-09-07', '2023-09-12', '2023-09-12', '2023-09-17', '2023-09-17', '2023-09-22', '2023-09-22', '2023-09-27', '2023-09-27', '2023-10-07', '2023-10-07']



In [ ]:
Map = geemap.Map()

Map.add_basemap("SATELLITE")

# Viz = {'min': 0, 'max': 5000, 'palette': [    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
#                                          '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
#                                          '012E01', '011D01', '011301']}

Map.addLayer(coll.first().select(['B4','B3','B2']), {}, "img1")

Map.centerObject(extractor.fc)
Map